In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
from pyomo.environ import (
    Constraint,
    Var,
    ConcreteModel,
    Expression,
    Objective,
    TransformationFactory,
    value,
    units as pyunits,
)
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock,
)
from idaes.models.properties.modular_properties.base.generic_reaction import (
    GenericReactionParameterBlock,
)
from idaes.models.unit_models import (
    Feed,
    Mixer,
    Compressor,
    Heater,
    EquilibriumReactor,
    Product,
)

from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

In [3]:
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop
import msr_reaction as reaction_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

In [5]:
thermo_props_config_dict = get_prop(components=["CH4", "H2O", "H2", "CO", "CO2"])
m.fs.thermo_params = GenericParameterBlock(**thermo_props_config_dict)
m.fs.reaction_params = GenericReactionParameterBlock(
    property_package=m.fs.thermo_params, **reaction_props.config_dict
)

In [6]:
m.fs.CH4 = Feed(property_package=m.fs.thermo_params)
m.fs.H2O = Feed(property_package=m.fs.thermo_params)
m.fs.PROD = Product(property_package=m.fs.thermo_params)
m.fs.M101 = Mixer(
    property_package=m.fs.thermo_params, inlet_list=["methane_feed", "steam_feed"]
)
m.fs.H101 = Heater(
    property_package=m.fs.thermo_params,
    has_pressure_change=False,
    has_phase_equilibrium=False,
)
m.fs.C101 = Compressor(property_package=m.fs.thermo_params)

In [7]:
m.fs.R101 = EquilibriumReactor(
    property_package=m.fs.thermo_params,
    reaction_package=m.fs.reaction_params,
    has_equilibrium_reactions=True,
    has_rate_reactions=False,
    has_heat_of_reaction=True,
    has_heat_transfer=True,
    has_pressure_change=False,
    has_phase_equilibrium=True,
)

In [8]:
m.fs.s01 = Arc(source=m.fs.CH4.outlet, destination=m.fs.M101.methane_feed)
m.fs.s02 = Arc(source=m.fs.H2O.outlet, destination=m.fs.M101.steam_feed)
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.C101.inlet)
m.fs.s04 = Arc(source=m.fs.C101.outlet, destination=m.fs.H101.inlet)
m.fs.s05 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s06 = Arc(source=m.fs.R101.outlet, destination=m.fs.PROD.inlet)

In [9]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [10]:
m.fs.hyd_prod = Expression(
    expr=pyunits.convert(
        m.fs.PROD.inlet.flow_mol[0]
        * m.fs.PROD.inlet.mole_frac_comp[0, "H2"]
        * m.fs.thermo_params.H2.mw,  # MW defined in properties as kg/mol
        to_units=pyunits.Mlb / pyunits.yr,
    )
)  # converting kg/s to MM lb/year

In [11]:
m.fs.cooling_cost = Expression(
    expr=2.12e-8 * (m.fs.R101.heat_duty[0])
)  # the reaction is endothermic, so R101 duty is positive
m.fs.heating_cost = Expression(
    expr=2.2e-7 * m.fs.H101.heat_duty[0]
)  # the stream must be heated to T_rxn, so H101 duty is positive
m.fs.compression_cost = Expression(
    expr=1.2e-6 * m.fs.C101.work_isentropic[0]
)  # the stream must be pressurized, so the C101 work is positive
m.fs.operating_cost = Expression(
    expr=(3600 * 8000 * (m.fs.heating_cost + m.fs.cooling_cost + m.fs.compression_cost))
)

In [12]:
print(degrees_of_freedom(m))

20


In [13]:
m.fs.CH4.outlet.mole_frac_comp[0, "CH4"].fix(1)
m.fs.CH4.outlet.mole_frac_comp[0, "H2O"].fix(1e-5)
m.fs.CH4.outlet.mole_frac_comp[0, "H2"].fix(1e-5)
m.fs.CH4.outlet.mole_frac_comp[0, "CO"].fix(1e-5)
m.fs.CH4.outlet.mole_frac_comp[0, "CO2"].fix(1e-5)
m.fs.CH4.outlet.flow_mol.fix(75 * pyunits.mol / pyunits.s)
m.fs.CH4.outlet.temperature.fix(298.15 * pyunits.K)
m.fs.CH4.outlet.pressure.fix(1e5 * pyunits.Pa)

m.fs.H2O.outlet.mole_frac_comp[0, "CH4"].fix(1e-5)
m.fs.H2O.outlet.mole_frac_comp[0, "H2O"].fix(1)
m.fs.H2O.outlet.mole_frac_comp[0, "H2"].fix(1e-5)
m.fs.H2O.outlet.mole_frac_comp[0, "CO"].fix(1e-5)
m.fs.H2O.outlet.mole_frac_comp[0, "CO2"].fix(1e-5)
m.fs.H2O.outlet.flow_mol.fix(234 * pyunits.mol / pyunits.s)
m.fs.H2O.outlet.temperature.fix(373.15 * pyunits.K)
m.fs.H2O.outlet.pressure.fix(1e5 * pyunits.Pa)

In [14]:
m.fs.C101.outlet.pressure.fix(pyunits.convert(2 * pyunits.bar, to_units=pyunits.Pa))
m.fs.C101.efficiency_isentropic.fix(0.90)
m.fs.H101.outlet.temperature.fix(500 * pyunits.K)

In [15]:
m.fs.R101.conversion = Var(
    initialize=0.80, bounds=(0, 1), units=pyunits.dimensionless
)  # fraction

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion
    * m.fs.R101.inlet.flow_mol[0]
    * m.fs.R101.inlet.mole_frac_comp[0, "CH4"]
    == (
        m.fs.R101.inlet.flow_mol[0] * m.fs.R101.inlet.mole_frac_comp[0, "CH4"]
        - m.fs.R101.outlet.flow_mol[0] * m.fs.R101.outlet.mole_frac_comp[0, "CH4"]
    )
)

m.fs.R101.conversion.fix(0.80)

In [16]:
print(degrees_of_freedom(m))

0


In [17]:
# Initialize and solve each unit operation
m.fs.CH4.initialize()
propagate_state(arc=m.fs.s01)

m.fs.H2O.initialize()
propagate_state(arc=m.fs.s02)

m.fs.M101.initialize()
propagate_state(arc=m.fs.s03)

m.fs.C101.initialize()
propagate_state(arc=m.fs.s04)

m.fs.H101.initialize()
propagate_state(arc=m.fs.s05)

m.fs.R101.initialize()
propagate_state(arc=m.fs.s06)

m.fs.PROD.initialize()

# set solver
solver = get_solver()

2023-02-18 12:07:08 [INFO] idaes.init.fs.CH4.properties: Starting initialization


2023-02-18 12:07:08 [INFO] idaes.init.fs.CH4.properties: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:07:08 [INFO] idaes.init.fs.CH4.properties: Property package initialization: optimal - Optimal Solution Found.


2023-02-18 12:07:08 [INFO] idaes.init.fs.CH4: Initialization Complete.


2023-02-18 12:07:08 [INFO] idaes.init.fs.H2O.properties: Starting initialization


...

2023-02-18 12:07:10 [INFO] idaes.init.fs.R101: Initialization Complete: optimal - Optimal Solution Found


2023-02-18 12:07:10 [INFO] idaes.init.fs.PROD.properties: Starting initialization


2023-02-18 12:07:10 [INFO] idaes.init.fs.PROD.properties: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:07:10 [INFO] idaes.init.fs.PROD.properties: Property package initialization: optimal - Optimal Solution Found.


2023-02-18 12:07:10 [INFO] idaes.init.fs.PROD: Initialization Complete.


In [18]:
# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [19]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

operating cost = $45.933 million per year


In [20]:
m.fs.R101.report()

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value      : Units : Fixed : Bounds
    Heat Duty : 2.7605e+07 :  watt : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                Units         Inlet     Outlet  
    Total Molar Flowrate     mole / second     309.01     429.02
    Total Mole Fraction CH4  dimensionless    0.24272   0.034965
    Total Mole Fraction H2O  dimensionless    0.75725    0.31487
    Total Mole Fraction H2   dimensionless 9.9996e-06    0.51029
    Total Mole Fraction CO   dimensionless 9.9996e-06   0.049157
    Total Mole Fraction CO2  dimensionless 9.9996e-06   0.090717
    Temperature                     kelvin     500.00     868.56
    Pressure                        pascal 2.0000e+05 2.0

In [21]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

In [22]:
m.fs.R101.conversion.fix(0.90)

m.fs.C101.outlet.pressure.unfix()
m.fs.C101.outlet.pressure[0].setlb(
    pyunits.convert(2 * pyunits.bar, to_units=pyunits.Pa)
)  # pressurize to at least 2 bar
m.fs.C101.outlet.pressure[0].setub(
    pyunits.convert(10 * pyunits.bar, to_units=pyunits.Pa)
)  # at most, pressurize to 10 bar

m.fs.H101.outlet.temperature.unfix()
m.fs.H101.heat_duty[0].setlb(
    0 * pyunits.J / pyunits.s
)  # outlet temperature is equal to or greater than inlet temperature
m.fs.H101.outlet.temperature[0].setub(1000 * pyunits.K)  # at most, heat to 1000 K

In [23]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://www.hsl.rl.ac.

In [24]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

print()
print("Compressor results")

m.fs.C101.report()

print()
print("Heater results")

m.fs.H101.report()

print()
print("Equilibrium reactor results")

m.fs.R101.report()

operating cost = $43.309 million per year

Compressor results

Unit : fs.C101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value      : Units         : Fixed : Bounds
    Isentropic Efficiency :    0.90000 : dimensionless :  True : (None, None)
          Mechanical Work : 7.5471e+05 :          watt : False : (None, None)
          Pressure Change : 1.0000e+05 :        pascal : False : (None, None)
           Pressure Ratio :     2.0000 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                Units         Inlet     Outlet  
    Total Molar Flowrate     mole / second     309.01     309.01
    Total Mole Fraction CH4  dimensionless    0.24272    0.24272
    Total Mole Fraction H2O  dimensionless    

In [25]:
print("Optimal Values")
print()

print(f"C101 outlet pressure = {value(m.fs.C101.outlet.pressure[0])/1E6:0.3f} MPa")
print()

print(f"C101 outlet temperature = {value(m.fs.C101.outlet.temperature[0]):0.3f} K")
print()

print(f"H101 outlet temperature = {value(m.fs.H101.outlet.temperature[0]):0.3f} K")

print()
print(f"R101 outlet temperature = {value(m.fs.R101.outlet.temperature[0]):0.3f} K")

print()
print(f"Hydrogen produced = {value(m.fs.hyd_prod):0.3f} MM lb/year")

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")

Optimal Values

C101 outlet pressure = 0.200 MPa

C101 outlet temperature = 423.345 K

H101 outlet temperature = 423.345 K

R101 outlet temperature = 910.044 K

Hydrogen produced = 33.648 MM lb/year

Conversion achieved = 90.0%
